In [1]:
import pandas as pd
import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import joblib
from tqdm import tqdm
import matplotlib.pyplot as plt
from copy import deepcopy

In [2]:
from isaac.utils import get_cuda_device_if_available
device = get_cuda_device_if_available()
print(device)

cuda:0


In [3]:
from isaac.dataset import read_dataset, prepare_dataset, sample_subsequences
from isaac.utils import plot_confusion_matrix
from isaac.models import ComplexRNNModel, initialise_model
from isaac.constants import *
from isaac.training import evaluate, training_loop, evaluate_saved_model
from isaac.sanity import class_proportions

In [4]:
BATCH_SIZE = 128
EPOCHS = 100
NORMALISE_DATA = True
SEQ_END = 1800

# Define common model, loss and optimizer

In [5]:
ALL_COLS = BASIC_TRAINING_COLS + PUCK_SQUARE_DISTANCES + PUCK_ANGLE_FEATURES

train_trials = read_dataset("data/train_passive_trials.h5", n_trials=3500, cols=ALL_COLS)
val_trials = read_dataset("data/val_passive_trials.h5", n_trials=800, cols=ALL_COLS)

100%|██████████| 10/10 [00:00<00:00, 122.39it/s]


In [6]:
question_type = "mass"

In [7]:
if question_type == "force":    
    class_columns = FORCE_CLASS_COLS
    TR_COLS = BASIC_TRAINING_COLS + PUCK_SQUARE_DISTANCES + PUCK_ANGLE_FEATURES
    STEP_SIZE = 3
else:
    class_columns = MASS_CLASS_COLS
    TR_COLS = BASIC_TRAINING_COLS
    STEP_SIZE = 4

In [8]:
INPUT_DIM = len(TR_COLS)    # input dimension
HIDDEN_DIM = 25  # hidden layer dimension
N_LAYERS = 4     # number of hidden layers
OUTPUT_DIM = 3   # output dimension
DROPOUT = 0.5

network_params = (INPUT_DIM, HIDDEN_DIM, N_LAYERS, OUTPUT_DIM, DROPOUT)

# TRAINING

In [9]:
stats_dfs = []

loaders, scaler = prepare_dataset([train_trials, val_trials], class_columns=class_columns, 
                                  training_columns=TR_COLS, batch_size=BATCH_SIZE, 
                                  normalise_data=NORMALISE_DATA, device=device)
best_model_overall = None
best_accuracy = 0.

for seed in [0, 42, 72]:
    df = pd.DataFrame(columns=["seed", "Epoch", "Loss"])


    model, error, optimizer = initialise_model(network_params, lr=0.01, seed=seed, device=device)
    epoch_losses, epoch_accuracies, best_model = training_loop(model, optimizer, error, loaders[0], 
                                                               loaders[1], EPOCHS, seq_end=SEQ_END, 
                                                               step_size=STEP_SIZE)

    model_accuracy = max(epoch_accuracies[1])
    if model_accuracy > best_accuracy:
        best_model_overall = deepcopy(best_model)
        best_accuracy = model_accuracy
    
    df["Epoch"] = np.arange(EPOCHS)
    df["Loss"] = epoch_losses
    df["Train Accuracy"] = epoch_accuracies[0]
    df["Val Accuracy"] = epoch_accuracies[1]
    df["seed"] = str(seed)
    stats_dfs.append(df)
        
stats = pd.concat(stats_dfs)

100%|██████████| 10/10 [00:00<00:00, 1096.92it/s]
Train_loss (1.11)	 Train_acc (80.00)	 Val_acc (50.00): 100%|██████████| 2/2 [00:00<00:00,  5.35it/s] 
Train_loss (1.14)	 Train_acc (50.00)	 Val_acc (30.00): 100%|██████████| 2/2 [00:00<00:00,  5.44it/s] 
Train_loss (1.15)	 Train_acc (40.00)	 Val_acc (40.00): 100%|██████████| 2/2 [00:00<00:00,  5.90it/s] 


In [10]:
best_accuracy

50.0

In [11]:
stats.to_hdf("final_choice_plots/"+question_type+"_stats.h5", key="stats")

## Save model and scaler

In [12]:
torch.save(best_model_overall.state_dict(), "models/passive_"+question_type+"_model.pt")
joblib.dump(scaler, "scalers/passive_"+question_type+"_scaler.sk")

['scalers/passive_mass_scaler.sk']